#Connect to sharepoint

In [1]:
!pip install office365
!pip install Office365-REST-Python-Client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 6.8 MB/s eta 0

In [2]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.files.file import File

####inputs########
# This will be the URL that points to your sharepoint site.
# Make sure you change only the parts of the link that start with "Your"
url_shrpt = 'https://ihuedu.sharepoint.com/sites/EDYTEProject2023/'
username_shrpt = '###############'
password_shrpt = '###############'
folder_files_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/raw_Data/'
folder_analysis_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/analysis_exports/'

In [3]:
###Authentication###For authenticating into your sharepoint site###
ctx_auth = AuthenticationContext(url_shrpt)
if ctx_auth.acquire_token_for_user(username_shrpt, password_shrpt):
  ctx = ClientContext(url_shrpt, ctx_auth)
  web = ctx.web
  ctx.load(web)
  ctx.execute_query()
  print('Authenticated into sharepoint as: ',web.properties['Title'])

else:
  print(ctx_auth.get_last_error())
############################

Authenticated into sharepoint as:  EDYTE Project 2023


In [4]:
####Function for extracting the file names of a folder in sharepoint###
###If you want to extract the folder names instead of file names, you have to change "sub_folders = folder.files" to "sub_folders = folder.folders" in the below function

global print_folder_contents
def print_folder_contents(ctx, folder_url):
    try:

        folder = ctx.web.get_folder_by_server_relative_url(folder_url)
        fold_names = []
        sub_folders = folder.files #Replace files with folders for getting list of folders
        ctx.load(sub_folders)
        ctx.execute_query()

        for s_folder in sub_folders:

            fold_names.append(s_folder.properties["Name"])

        return fold_names

    except Exception as e:
        print('Problem printing out library contents: ', e)
######################################################

# Call the function by giving your folder URL as input
filelist_shrpt=print_folder_contents(ctx,folder_files_url_shrpt)
#Print the list of files present in the folder
print(filelist_shrpt)

['process-provision-digital-locations.csv', 'process-steps-digital.csv', 'process-steps.csv', 'process-evidences-cost.csv', 'process.csv', 'process-rules.csv', 'process-conditions.csv', 'process-evidences.csv']


# Import Data

In [5]:
import pandas as pd
import csv
import numpy as np
import io
import os
import tempfile

In [6]:
def process_csv_file(file_name, column_names):
    file_url= folder_files_url_shrpt+file_name
    response = File.open_binary(ctx, file_url)  # Assuming File is imported and ctx is defined

    df = pd.read_csv(io.BytesIO(response.content))
    df = df.loc[:, column_names]
    df = df.dropna(subset=column_names[-1])

    return df

> process description

> process remarks

> process steps description

> steps digital description

> evidences

> conditions

In [7]:
#process description
process_description = process_csv_file("process.csv", ['id', 'title_el', 'description'])

#process remarks
process_remarks = process_csv_file("process.csv", ['id','title_el', 'remarks'])

#steps description
process_steps = process_csv_file("process-steps.csv", ['process_id', "ihu_unique_step_id", 'step_title', 'step_description'])

#digital steps description
process_steps_digital = process_csv_file("process-steps-digital.csv", ['process_id', "ihu_unique_step_digital_id", 'step_digital_title', 'step_digital_description'])

#evidences description
process_evidences = process_csv_file("process-evidences.csv", ['process_id', 'ihu_unique_evidence_id', 'evidence_description'])

#conditions name
process_conditions = process_csv_file("process-conditions.csv", ['process_id', 'ihu_unique_condition_id', 'conditions_name'])

# Import Models

In [14]:
!pip install transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=c5e4f33d2223f619be2235b6ad85885d9b90f9aa220cc9b95fb0771e4418553b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [15]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import re

#NER Model

In [16]:
ner_model = AutoModelForTokenClassification.from_pretrained("amichailidis/bert-base-greek-uncased-v1-finetuned-ner")
ner_pip = pipeline("ner", model=ner_model, tokenizer= ('amichailidis/bert-base-greek-uncased-v1-finetuned-ner', {'model_max_length': 512}), grouped_entities=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [17]:
def extract_entities(df, column_name, id):
    # Define the regex patterns
    column = column_name
    id = id
    regex_pattern = "\s*\([^)]*\)" # remove parenthesis

    extracted_entities = []
    excluded_words = ["τμημα", "τμηματος", "υπηρεσια", "υπηρεσιας", "υπαλληλος", "απο", "αιτουντος", "σε"]
    excluded_patterns = ["##"]

    # Iterate over the 'step_description' column
    for index, row in df.iterrows():
        target_column = row[column]

        # Apply the first regex pattern
        target_column = re.sub(regex_pattern, "", target_column)
        target_column = re.sub(r"-", " ", target_column)

        # Use the NER model to extract entities
        entities = ner_pip(target_column)

        for entity in entities:
            word = entity['word']
            # Check if the word is in the excluded list
            if word not in excluded_words and all(pattern not in word for pattern in excluded_patterns) and len(word) > 1:
                extracted_entities.append({id: row[id], 'word': word})

    results = pd.DataFrame(extracted_entities)
    return results

# Run the process

In [20]:
def upload_csv_to_target_folder(results, name):

  path = name+".csv"

  results.to_csv(path, index=False)

  url=folder_analysis_url_shrpt+"entities_extraction/"+"entities_extracted"

  target_folder = ctx.web.get_folder_by_server_relative_url(url)
  with open(path, "rb") as content_file:
      file_content = content_file.read()
      target_folder.upload_file(os.path.basename(path), file_content).execute_query()

> process_description

id

description

In [18]:
results_process_description = extract_entities(process_description, "description", "id")
upload_csv_to_target_folder(results_process_description, "process_description")

> process_remarks

id

remarks::

In [22]:
results_process_remarks = extract_entities(process_remarks, "remarks", "id")
upload_csv_to_target_folder(results_process_remarks, "process_remarks")

> process_steps

ihu_unique_step_id

step_description

In [23]:
results_process_steps = extract_entities(process_steps, "step_description", "ihu_unique_step_id")
upload_csv_to_target_folder(results_process_steps, "process_steps")

> process_steps_digital

ihu_unique_step_digital_id

step_digital_description


In [25]:
results_process_steps_digital = extract_entities(process_steps_digital, "step_digital_description", "ihu_unique_step_digital_id")
upload_csv_to_target_folder(results_process_steps_digital, "process_steps_digital")

> process_evidences

unique_evidence_id

evidence_description


In [ ]:
results_process_evidences = extract_entities(process_evidences, "evidence_description", "ihu_unique_evidence_id")
upload_csv_to_target_folder(results_process_evidences, "process_evidences")

> process_conditions

unique_condition_id

conditions_name

In [ ]:
results_process_conditions = extract_entities(process_conditions, "conditions_name", "ihu_unique_condition_id")
upload_csv_to_target_folder(results_process_conditions, "process_conditions")